## 语言模型language model构建训练数据

1. 打开文件
2. 非字母都用空格代替
3. 两端的空格去掉
4. 大写转小写

lines：(list，1D)

多行文本数据，每个元素为一行str，str里只有单词和空格

In [8]:
import os
import re

data_dir='./../data'
with open(os.path.join(data_dir, 'timemachine.txt'), 'r',encoding='utf-8') as f:
    #一次性多行读取，list，每个元素是一行str
    lines = f.readlines()
    
    lines = [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]
#字符串
print(type(lines))
print(lines[:75])

<class 'list'>
['the time machine by h g wells', '', '', '', '', 'i', '', '', 'the time traveller for so it will be convenient to speak of him', 'was expounding a recondite matter to us his grey eyes shone and', 'twinkled and his usually pale face was flushed and animated the', 'fire burned brightly and the soft radiance of the incandescent', 'lights in the lilies of silver caught the bubbles that flashed and', 'passed in our glasses our chairs being his patents embraced and', 'caressed us rather than submitted to be sat upon and there was that', 'luxurious after dinner atmosphere when thought roams gracefully', 'free of the trammels of precision and he put it to us in this', 'way marking the points with a lean forefinger as we sat and lazily', 'admired his earnestness over this new paradox as we thought it', 'and his fecundity', '', 'you must follow me carefully i shall have to controvert one or two', 'ideas that are almost universally accepted the geometry for', 'instance they taught

### 将1D的list差分为2D的list
内部从str拆分为token级别

In [10]:
def tokenize(lines, token='word'):
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)


word_tokens = tokenize(lines)
for i in range(11):
    print(word_tokens[i])

char_tokens = tokenize(lines,'char')
for i in range(11):
    print(char_tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']
[]
[]
[]
[]
['i']
[]
[]
['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 't', 'r', 'a', 'v', 'e', 'l', 'l', 'e', 'r', ' ', 'f', 'o', 'r', ' ', 's', 'o', ' ', 'i', 't', ' ', 'w', 'i', 'l', 'l', ' ', 'b', 'e', ' ', 'c', 'o', 'n', 'v', 'e', 'n', 'i', 'e', 'n', 't', ' ', 't', 'o', ' ', 's', 'p', 'e', 'a', 'k', ' ', 'o', 'f', ' ', 'h', 'i', 'm']
['w', 'a', 's', ' ', 'e', 'x', 'p', 'o', 'u', 'n', 'd', 'i', 'n', 'g', ' ', 'a', ' ', 'r', 'e', 'c', 'o', 'n', 'd', 'i', 't', 'e', ' ',

## 准备序列训练数据
- copus：语料，就是数据集内容，里面是token或者token索引
- batch_size:批量
- num_steps:序列长度

采样有3个点：
1. 随机采样和顺序采样，意思是一个batch里的数据是否是顺序的
2. 初始采样的位置要偏移一下

### 语言模型数据特点
- 本质是$x_{t-1}$预测$x_t$
- 所以数据里y是下一个x，y序列正好是x序列向右偏移一个位置

In [ ]:
def seq_data_iter_random(corpus, batch_size, num_steps):  #@save
    """使用随机抽样生成一个小批量子序列"""
    # 从随机偏移量开始对序列进行分区，随机范围包括num_steps-1
    corpus = corpus[random.randint(0, num_steps - 1):]
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)

def seq_data_iter_sequential(corpus, batch_size, num_steps):  #@save
    """使用顺序分区生成一个小批量子序列"""
    # 从随机偏移量开始划分序列
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    Xs = torch.tensor(corpus[offset: offset + num_tokens])
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y